
# פרויקט טיטאניק — זרימת למידת מכונה (Supervised Learning Flow)

## הקדמה
בתרגיל זה אנו עוסקים בבעיה של **למידה מונחית מסוג סיווג (Classification)**.
המטרה היא לחזות האם נוסע בטיטאניק שרד (`Survived = 1`) או לא שרד (`Survived = 0`).
מדד האיכות המרכזי – **F1-score** לבעיה בינארית (עם מחלקה מרכזית אחת).



## ספריות ושגרות עזר
נייבא ספריות לעיבוד נתונים, ויזואליזציה ומודלים. נגדיר פונקציות עזר לשכפול קוד.


In [ ]:

# ספריות בסיסיות
import pandas as pd
import numpy as np

# ויזואליזציה
import matplotlib.pyplot as plt

# מודלים וכלי עזר מ-sklearn
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import f1_score, classification_report, ConfusionMatrixDisplay
from sklearn.impute import SimpleImputer

# אלגוריתמים
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# תצוגה
pd.set_option('display.max_columns', 100)

def plot_hist(series, title):
    plt.figure()
    series.dropna().hist(bins=30)
    plt.title(title)
    plt.xlabel(series.name)
    plt.ylabel('Count')
    plt.show()



## טעינת הנתונים
בשלב זה אנו טוענים את קובצי האימון והבדיקה (`titanic_train.csv`, `titanic_test.csv`).
נציג מספר שורות ראשונות, ממדי הדאטה, וסיכום מידע.
> שימו לב: עדכנו את המסלול לקבצים בהתאם למיקום המקומי אצלכם אם צריך.


In [ ]:

# נתיבים יחסיים בתוך התיקייה
train_path = 'titanic_train.csv'
test_path  = 'titanic_test.csv'

df_train = pd.read_csv(train_path)
df_test  = pd.read_csv(test_path)

print('Train shape:', df_train.shape)
print('Test shape :', df_test.shape)

df_train.head()


In [ ]:

# מידע כללי וערכים חסרים
display(df_train.info())
display(df_train.isna().sum().sort_values(ascending=False).to_frame('missing_train'))
display(df_test.isna().sum().sort_values(ascending=False).to_frame('missing_test'))



## ניתוח ראשוני (EDA)
בבדיקה ראשונית נתמקד במשתנים רלוונטיים כמו גיל (Age), מחיר כרטיס (Fare), מחלקה (Pclass), מין (Sex), ונראה קשרים אפשריים להישרדות.


In [ ]:

# התפלגות גיל
if 'Age' in df_train.columns:
    plot_hist(df_train['Age'], 'התפלגות גיל (Train)')
# התפלגות מחיר כרטיס
if 'Fare' in df_train.columns:
    plot_hist(df_train['Fare'], 'התפלגות מחיר כרטיס (Train)')


In [ ]:

# שיעור הישרדות לפי מגדר ומחלקה (אם קיימים)
eda_tbls = {}
if {'Sex','Survived'}.issubset(df_train.columns):
    eda_tbls['Survival by Sex'] = df_train.pivot_table(index='Sex', values='Survived', aggfunc='mean')
if {'Pclass','Survived'}.issubset(df_train.columns):
    eda_tbls['Survival by Pclass'] = df_train.pivot_table(index='Pclass', values='Survived', aggfunc='mean')
for k, v in eda_tbls.items():
    print(k)
    display(v)



## הנדסת מאפיינים (Feature Engineering)
- השלמת ערכים חסרים עבור `Age`, `Embarked` (ועוד לפי הצורך).
- המרת משתנים קטגוריים ל-One-Hot.
- יצירת מאפיינים חדשים לדוגמה:
  - `FamilySize = SibSp + Parch + 1`
  - `IsAlone = (FamilySize == 1)`
  - `Deck` – האות הראשונה מתוך Cabin (אם קיים)


In [ ]:

def add_engineered_features(df):
    df = df.copy()
    # גודל משפחה
    if set(['SibSp','Parch']).issubset(df.columns):
        df['FamilySize'] = df['SibSp'].fillna(0) + df['Parch'].fillna(0) + 1
        df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    # סיפון מתוך Cabin
    if 'Cabin' in df.columns:
        df['Deck'] = df['Cabin'].astype(str).str[0].replace('n', np.nan)
    return df

df_train_fe = add_engineered_features(df_train)
df_test_fe  = add_engineered_features(df_test)

df_train_fe.head()



## הכנת צינור עיבוד (Preprocessing) ומודלים
נגדיר עמודות נומריות וקטגוריאליות, ונבנה `ColumnTransformer` עם איפיונון/סטנדרטיזציה וקידוד One-Hot.
לאחר מכן נגדיר מספר מודלים ונריץ Grid Search עם 5-Fold CV.
מדד איכות: **F1 (binary)**.


In [ ]:

target_col = 'Survived'
id_col = 'PassengerId' if 'PassengerId' in df_train_fe.columns else None

# בחירת עמודות
X = df_train_fe.drop(columns=[target_col])
y = df_train_fe[target_col]

# עמודות נומריות וקטגוריאליות
numeric_features = [c for c in X.columns if np.issubdtype(X[c].dtype, np.number)]
categorical_features = [c for c in X.columns if c not in numeric_features]

# מסירים מזהים שאינם תורמים (אם קיימים)
for col in ['PassengerId','Ticket']:
    if col in numeric_features:
        numeric_features.remove(col)
    if col in categorical_features:
        categorical_features.remove(col)

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# מודלים
pipe_logreg = Pipeline(steps=[('preprocess', preprocess),
                              ('model', LogisticRegression(max_iter=1000))])

pipe_tree = Pipeline(steps=[('preprocess', preprocess),
                            ('model', DecisionTreeClassifier(random_state=42))])

pipe_rf = Pipeline(steps=[('preprocess', preprocess),
                          ('model', RandomForestClassifier(random_state=42))])

# גרידים
param_grid_logreg = {
    'model__C': [0.1, 1.0, 3.0],
    'model__penalty': ['l2'],
    'model__solver': ['lbfgs', 'liblinear']
}

param_grid_tree = {
    'model__max_depth': [3, 5, 8, None],
    'model__min_samples_split': [2, 5, 10]
}

param_grid_rf = {
    'model__n_estimators': [100, 300],
    'model__max_depth': [None, 5, 8],
    'model__min_samples_split': [2, 5]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grids = [
    ('LogisticRegression', pipe_logreg, param_grid_logreg),
    ('DecisionTree', pipe_tree, param_grid_tree),
    ('RandomForest', pipe_rf, param_grid_rf)
]

results = []
best_estimators = {}

for name, pipe, grid in grids:
    gs = GridSearchCV(
        estimator=pipe,
        param_grid=grid,
        scoring='f1',
        cv=cv,
        n_jobs=-1,
        refit=True,
        verbose=0
    )
    gs.fit(X, y)
    results.append({
        'model': name,
        'best_score_mean_cv_f1': gs.best_score_,
        'best_params': gs.best_params_
    })
    best_estimators[name] = gs.best_estimator_

res_df = pd.DataFrame(results).sort_values('best_score_mean_cv_f1', ascending=False)
res_df



## בחירת המודל הטוב ביותר ואימון על כל קבוצת האימון
נבחר את המודל הטוב ביותר לפי ה־F1 הממוצע ב־CV, נאמן אותו (כבר מאומן ב־refit=True), ונבדוק את ביצועיו על קבוצת האימון (אזהרת הטיה אפשרית).


In [ ]:

best_row = res_df.iloc[0]
best_name = best_row['model']
best_model = best_estimators[best_name]

print('המודל הטוב ביותר:', best_name)
print('F1 ממוצע (CV):', best_row['best_score_mean_cv_f1'])
print('היפר-פרמטרים:', best_row['best_params'])

# הערכה על train (לצורכי ניטור בלבד; לא מהווה תחליף ל-test)
y_pred_train = best_model.predict(X)
print(classification_report(y, y_pred_train, digits=4))



## חיזוי על קבוצת הבדיקה (Test Set) והפקת קובץ תוצאות
נחזה על סט הבדיקה ונפיק קובץ CSV עם `PassengerId` ו־`Survived`.


In [ ]:

# הכנת test לחיזוי
X_test = df_test_fe.copy()

# אם PassengerId קיים, נשמור אותו לייצוא
pid = X_test['PassengerId'] if 'PassengerId' in X_test.columns else pd.Series(range(1, len(X_test)+1))

# התאמת עמודות
missing_cols = [c for c in X.columns if c not in X_test.columns]
for c in missing_cols:
    X_test[c] = np.nan
extra_cols = [c for c in X_test.columns if c not in X.columns]
X_test = X_test[X.columns]

# חיזוי
test_pred = best_model.predict(X_test)

# יצוא
sub = pd.DataFrame({'PassengerId': pid, 'Survived': test_pred.astype(int)})
sub_path = 'titanic_submission.csv'
sub.to_csv(sub_path, index=False)
sub.head()



## (רשות) מטריצת בלבול על קבוצת האימון
להדגמה ויזואלית של הביצועים (שימו לב: זו לא הערכת ביצועים על Test).


In [ ]:

try:
    fig, ax = plt.subplots()
    ConfusionMatrixDisplay.from_predictions(y, y_pred_train, ax=ax)
    ax.set_title('Confusion Matrix (Train)')
    plt.show()
except Exception as e:
    print('לא ניתן לצייר מטריצה:', e)



## סיכום
בפרויקט זה יישמנו זרימת למידת מכונה מונחית לבעיית סיווג טיטאניק:
טעינת נתונים, EDA, הנדסת מאפיינים, הכנת צינור עיבוד, ניסויי מודלים עם Grid Search ו־5-Fold CV,
בחירת המודל הטוב ביותר, חיזוי על קבוצת הבדיקה והפקת קובץ תוצאות.
